In [2]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [5]:
import numpy as np

twod = np.array([[1, 2, 3], [3, 4, 5]])
for row, col in twod:
  print(row, col)

ValueError: too many values to unpack (expected 2)

In [1]:
import numpy as np
from datasets import load_dataset

from seq import Widget
from seq.data import get_featured_ids, get_ids, get_tokenizer
from seq.utils import cluster_sequences, mask_small_clusters, sort_sequences

In [2]:
ds = load_dataset("neuralbioinfo/bacterial_promoters")

In [3]:
tokenizer = get_tokenizer(type="dna", k=1)
ids, tokens = get_ids(
  ds["test_multispecies"]["segment"], tokenizer, max_tokens=32
)

100%|██████████| 22582/22582 [00:00<00:00, 39159.77it/s]


In [6]:
print(tokenizer._token_to_id)
print(tokenizer._id_to_token)

{'[MASK]': 0, 'C': 1, 'G': 2, 'A': 3, 'T': 4}
{0: '[MASK]', 1: 'C', 2: 'G', 3: 'A', 4: 'T'}


In [5]:
[0, 1, 2][1:1]

[]

In [4]:
featured_ids = get_featured_ids(ids, tokenizer, n_features=100, method="count")
labels = [{"id": i, "label": tokenizer.id_to_token(i)} for i in featured_ids]

In [37]:
import numpy as np
from collections import defaultdict
from typing import List, Tuple, Dict, Set


class Chunk:
  def __init__(
    self,
    subsequence: np.ndarray,
    start: int,
    end: int,
    sequence_indices: List[int],
  ):
    self.subsequence = subsequence
    self.start = start
    self.end = end
    self.sequence_indices = sequence_indices
    self.length = end - start

  def __eq__(self, other):
    return (
      np.array_equal(self.subsequence, other.subsequence)
      and self.start == other.start
      and self.end == other.end
      and self.sequence_indices == other.sequence_indices
    )

  def __hash__(self):
    return hash(
      (
        self.start,
        self.end,
        tuple(self.subsequence),
        tuple(self.sequence_indices),
      )
    )

  def __repr__(self):
    return f"Chunk({self.subsequence}, {self.start}, {self.end}, #{len(self.sequence_indices)})"


class ChunkDB:
  def __init__(
    self,
    sequences: np.ndarray,
    threshold: int = 10,
    max_chunk_len: int = 100000,
  ):
    self.sequences = sequences
    self.threshold = threshold
    self.max_chunk_len = max_chunk_len
    self.chunks: Set[Chunk] = set()
    self.start_index_map: Dict[int, List[Chunk]] = defaultdict(list)
    self.end_index_map: Dict[int, List[Chunk]] = defaultdict(list)
    self.start_end_index_map: Dict[Tuple[int, int], List[Chunk]] = defaultdict(
      list
    )
    self.start_subsequence_map: Dict[Tuple[int, str], List[Chunk]] = (
      defaultdict(list)
    )
    self.end_subsequence_map: Dict[Tuple[int, str], List[Chunk]] = defaultdict(
      list
    )

    self._initialize_chunks()

  def _initialize_chunks(self):
    n_sequences, n_length = self.sequences.shape
    used_positions = np.zeros((n_sequences, n_length), dtype=bool)
    max_chunk_len = min(self.max_chunk_len, n_length)

    for chunk_length in range(max_chunk_len, 0, -1):
      for start in range(n_length - chunk_length + 1):
        end = start + chunk_length

        # Extract valid sequences
        valid_sequences = ~used_positions[:, start:end].any(axis=1)
        if valid_sequences.sum() < self.threshold:
          continue

        # Hash chunks using start and end indices
        chunk_dict = defaultdict(list)
        for seq_idx, sequence in enumerate(
          self.sequences[valid_sequences, start:end]
        ):
          chunk_dict[tuple(sequence)].append(seq_idx)

        # Process unique chunks
        for subsequence, seq_indices in chunk_dict.items():
          if len(seq_indices) >= self.threshold:
            true_indices = np.arange(n_sequences)[valid_sequences][seq_indices]
            new_chunk = Chunk(
              np.array(subsequence), start, end, true_indices.tolist()
            )
            self._add_chunk(new_chunk)
            used_positions[true_indices, start:end] = True

  def _add_chunk(self, chunk: Chunk):
    self.chunks.add(chunk)

    self.start_index_map[chunk.start].append(chunk)
    self.end_index_map[chunk.end].append(chunk)
    self.start_end_index_map[(chunk.start, chunk.end)].append(chunk)

    subsequence_str = "".join(map(str, chunk.subsequence))
    for i in range(len(chunk.subsequence)):
      self.start_subsequence_map[(chunk.start, subsequence_str[:i])].append(
        chunk
      )
      self.end_subsequence_map[(chunk.end, subsequence_str[::-1][:i])].append(
        chunk
      )

  def get_chunks_by_start(self, start: int) -> List[Chunk]:
    return self.start_index_map[start]

  def get_chunks_by_end(self, end: int) -> List[Chunk]:
    return self.end_index_map[end]

  def get_chunks_by_start_end(self, start: int, end: int) -> List[Chunk]:
    return self.start_end_index_map[(start, end)]

  def get_chunks_by_start_and_subsequence(
    self, start: int, subsequence: np.ndarray
  ) -> List[Chunk]:
    subsequence_str = "".join(map(str, subsequence))
    return self.subsequence_map[(start, subsequence_str)]

  def get_chunks_by_end_and_subsequence(
    self, end: int, subsequence: np.ndarray
  ) -> List[Chunk]:
    subsequence_str = "".join(map(str, subsequence[::-1]))
    return self.subsequence_map[(end, subsequence_str)]


n_sequences, n_length = 10_000, 32
data = np.random.randint(0, 5, size=(n_sequences, n_length))
chunk_db = ChunkDB(data)
chunks_starting_at_1 = chunk_db.get_chunks_by_start(1)

In [39]:
chunk_db.get_chunks_by_end(1)

[Chunk([2], 0, 1, #21),
 Chunk([1], 0, 1, #28),
 Chunk([3], 0, 1, #37),
 Chunk([4], 0, 1, #35),
 Chunk([0], 0, 1, #44)]

In [29]:
from collections import defaultdict
from gettext import find
from re import sub
from typing import Dict, List, NamedTuple, Optional, Tuple

import numpy as np


class Chunk(NamedTuple):
  subsequence: np.ndarray
  start: int
  end: int
  seq_indices: List[int]

  def __eq__(self, other):
    if not isinstance(other, Chunk):
      return False
    return (
      self.start == other.start
      and self.end == other.end
      and np.array_equal(self.subsequence, other.subsequence)
    )

  def __hash__(self):
    return hash((self.start, self.end, tuple(self.subsequence)))


def find_unique_chunks(
  data: np.ndarray, threshold: int, max_chunk_len: int = 100000
):
  n_sequences, n_length = data.shape
  chunks = dict()
  used_positions = np.zeros((n_sequences, n_length), dtype=bool)
  max_chunk_len = min(max_chunk_len, n_length)

  for chunk_length in range(max_chunk_len, 0, -1):
    for start in range(n_length - chunk_length + 1):
      end = start + chunk_length

      # Extract valid sequences
      valid_sequences = ~used_positions[:, start:end].any(axis=1)
      if valid_sequences.sum() < threshold:
        continue

      # Hash chunks using start and end indices
      chunk_dict = defaultdict(list)
      for seq_idx, sequence in enumerate(data[valid_sequences, start:end]):
        chunk_dict[tuple(sequence)].append(seq_idx)
      # Process unique chunks
      for subsequence, seq_indices in chunk_dict.items():
        if len(seq_indices) >= threshold:
          true_indices = np.arange(n_sequences)[valid_sequences][seq_indices]
          new_chunk = Chunk(
            np.array(subsequence), start, end, true_indices.tolist()
          )
          chunks[new_chunk] = new_chunk
          # chunks.add(new_chunk)
          used_positions[true_indices, start:end] = True

  return chunks


# Example usage
n_sequences, n_length = 100_000, 32
data = np.random.randint(0, 5, size=(n_sequences, n_length))
threshold = 10
max_chunk_len = 5
unique_chunks = find_unique_chunks(data, threshold, max_chunk_len)
print(len(unique_chunks))


def find_subchunks(chunk: Chunk, chunks: dict[Chunk, Chunk]):
  length = chunk.end - chunk.start
  subchunks: list[Chunk] = []
  for i in range(length - 1, 0, -1):
    for start in range(chunk.start, chunk.end - i + 1):
      end = start + i
      subsequence = chunk.subsequence[start - chunk.start : end - chunk.start]
      subchunk = Chunk(subsequence, start, end, [])
      if subchunk in chunks:
        subchunks.append(chunks[subchunk])
    if len(subchunks) > 1:
      return sorted(subchunks, key=lambda x: len(x.seq_indices), reverse=True)
  return None

18775


In [18]:
list_chunks = list(unique_chunks)
list_chunks = sorted(
  list_chunks, key=lambda x: len(x.subsequence), reverse=True
)
print(len(list_chunks))

subchunks = [find_subchunks(chunk, unique_chunks) for chunk in list_chunks]
subchunks = [x for x in subchunks]
print(len(subchunks))
i = 100

print(list_chunks[i])
print(subchunks[i])

18775
18775
Chunk(subsequence=array([3, 1, 0, 0, 3]), start=0, end=5, seq_indices=[100, 385, 2597, 4121, 7124, 10924, 12278, 15909, 18928, 19072, 20527, 20999, 21881, 30176, 30681, 33489, 34713, 35160, 38605, 40615, 42982, 43314, 45677, 46970, 48870, 49758, 54085, 61184, 62688, 63460, 66030, 74745, 83324, 87129, 94007, 98067, 98533])
None


In [ ]:
import numpy as np
from collections import defaultdict
import networkx as nx


def hamming_distance(s1, s2):
  return sum(c1 != c2 for c1, c2 in zip(s1, s2))


def find_unique_chunks(sequences, elem_index, thresholds, max_window):
  n_sequences, n_length = sequences.shape
  result = defaultdict(list)
  processed_lengths = set()

  start = elem_index
  for window in range(max_window, 1, -1):
    end = start + window
    if end > n_length:
      continue

    if window in processed_lengths:
      continue

    chunks = [tuple(seq[start:end]) for seq in sequences]
    unique_chunks, counts = np.unique(chunks, axis=0, return_counts=True)

    for chunk, count in zip(unique_chunks, counts):
      if count >= thresholds:
        chunk_key = tuple(chunk)
        seq_ids = [
          i for i, seq_chunk in enumerate(chunks) if seq_chunk == chunk_key
        ]
        if chunk_key not in result:
          result[chunk_key] = seq_ids
          processed_lengths.add(window)

  return result


def sort_chunks_by_hamming_distance(chunks):
  G = nx.Graph()
  chunk_list = list(chunks.keys())

  for i in range(len(chunk_list)):
    for j in range(i + 1, len(chunk_list)):
      distance = hamming_distance(chunk_list[i], chunk_list[j])
      G.add_edge(i, j, weight=distance)

  # Solve TSP
  tsp = nx.approximation.christofides(G)

  # Reorder chunks based on TSP solution
  sorted_chunks = [(chunk_list[i], chunks[chunk_list[i]]) for i in tsp]

  return sorted_chunks


# 예시 사용
sequences = np.random.randint(0, 4, (1000, 20))  # 100개의 시퀀스, 각 20개 요소
elem_index = 5
thresholds = 3
max_window = 5

unique_chunks = find_unique_chunks(
  sequences, elem_index, thresholds, max_window
)

# 결과를 hamming distance에 따라 정렬
sorted_chunks = sort_chunks_by_hamming_distance(unique_chunks)

# 정렬된 결과 출력
for chunk, seq_ids in sorted_chunks:
  print(f"Chunk {chunk}: Sequences {seq_ids}")

In [19]:
from collections import defaultdict

import numpy as np


def count_unique_chunks(sequences: np.ndarray, chunk_size: int = 1):
  _, n_length = sequences.shape

  result = []
  for i in range(n_length - chunk_size + 1):
    chunks = sequences[:, i : i + chunk_size]
    unique_chunks = defaultdict(list)
    for index, chunk in enumerate(chunks):
      unique_chunks[tuple(chunk.tolist())].append(index)
    result.append(unique_chunks)

  return result


n_chunks = [count_unique_chunks(ids, i) for i in range(1, 10)]

In [31]:
for window in n_chunks[8]:
  for k, v in window.items():
    if len(v) > 3:
      print(k, v)

(2, 3, 4, 3, 4, 4, 4, 4, 3) [2391, 9009, 11110, 12278]
(4, 4, 2, 1, 1, 2, 2, 3, 4) [3301, 11351, 14479, 18473]
(2, 1, 2, 3, 4, 1, 2, 1, 4) [3628, 10271, 10468, 10718]
(3, 4, 4, 4, 4, 3, 3, 3, 3) [5070, 9696, 9709, 11168]
(1, 2, 1, 1, 4, 2, 1, 3, 2) [6173, 21647, 21740, 21934]
(1, 4, 2, 2, 3, 4, 4, 4, 4) [8132, 8665, 10459, 12874]
(1, 2, 2, 4, 1, 4, 1, 2, 1) [1649, 8881, 18176, 18637]
(4, 4, 4, 3, 4, 4, 4, 4, 4) [2199, 5013, 12448, 19329]
(4, 4, 4, 4, 3, 3, 3, 3, 3) [2644, 9696, 9709, 11168]
(4, 4, 3, 4, 4, 4, 4, 4, 4) [5051, 10290, 15460, 20010]
(4, 4, 4, 3, 3, 3, 3, 3, 4) [2221, 2644, 11168, 15570]
(4, 4, 4, 1, 3, 3, 3, 3, 3) [2223, 11302, 13753, 19088]
(4, 4, 4, 4, 2, 3, 3, 3, 3) [2272, 2301, 5545, 5556]
(4, 1, 4, 2, 4, 3, 3, 3, 4) [2546, 4079, 10270, 13498]
(1, 4, 1, 1, 2, 2, 1, 2, 4) [2830, 4403, 18432, 19833]
(4, 4, 3, 3, 3, 3, 3, 3, 3) [5080, 5685, 5943, 11202, 12311]
(1, 2, 3, 1, 2, 1, 2, 3, 4) [6492, 7444, 11580, 18858]
(4, 2, 4, 3, 3, 3, 3, 3, 3) [9545, 9729, 9776, 11366]
(4, 

In [23]:
w = Widget(
  sequences=ids,
  labels=labels[0:10],
  width=800,
  height=400,
  grid=False,
)
init_seq = w.sequences
w

Sequences:  (22506, 32)
Rects:  101339


Widget(height=400, labels=[{'id': 53, 'label': 'GCG'}, {'id': 44, 'label': 'TTT'}, {'id': 64, 'label': 'AAA'},…

In [24]:
w.update_sequences(init_seq)
w.grid = False

Sequences:  (22506, 32)
Rects:  101339


In [25]:
clustered_sequence = cluster_sequences(init_seq)
w.update_sequences(clustered_sequence)

Sequences:  (22506, 32)
Rects:  69844


In [27]:
masked_sequences = mask_small_clusters(clustered_sequence, min_cluster_size=5)
w.grid = False
w.update_sequences(masked_sequences)

Sequences:  (22043, 32)
Rects:  2094


In [18]:
# sorted_sequences = sort_sequences(w.sequences)
# w.update_sequences(sorted_sequences)